# Sales Copilot

Additional concepts:

- Copilot determines intent. Either goes to load balance endpoint or Assistant depending on user intent.
- Thread management for multiple users
  - Using shelves to store the user threads
  - Thread will be created or reused as different users access the assistant
- Relationships in files

## Diagram

- **Note**: Not implemented, promptflow and weather services

![Architecture Diagram](../images/copilot-with-assistant.png)

### Load the required libraries

In [ ]:
import os
import assistant as sales
from pathlib import Path

from dotenv import load_dotenv
from openai import AzureOpenAI
from openai.types import FileObject

### Load the environment variables

In [ ]:
load_dotenv()
api_endpoint = os.getenv("OPENAI_URI")
api_key = os.getenv("OPENAI_KEY")
api_version = os.getenv("OPENAI_VERSION")
api_deployment_name = os.getenv("OPENAI_GPT_DEPLOYMENT")
email_URI = os.getenv("EMAIL_URI")

### Create the AzureOpenAI clients

In [ ]:
load_balanced_client = AzureOpenAI(api_key=api_key, api_version=api_version, azure_endpoint=api_endpoint)
assistant_client = AzureOpenAI(api_key=api_key, api_version=api_version, azure_endpoint=api_endpoint)

### Define the Assistant tools

In [ ]:
tools_list = [
    {"type": "code_interpreter"}
]

### Upload the file(s)

In [ ]:
DATA_FOLDER = "data/sales/"

def upload_file(client: AzureOpenAI, path: str) -> FileObject:
    print(path)
    with Path(path).open("rb") as f:
        return client.files.create(file=f, purpose="assistants")

arr = os.listdir(DATA_FOLDER)
assistant_files = []
for file in arr:
    filePath = DATA_FOLDER + file
    assistant_file = upload_file(assistant_client, filePath)
    sales.ai_files.append(assistant_file)
    assistant_files.append(assistant_file)

file_ids = [file.id for file in assistant_files]

### Create an Assistant and a Thread

In [ ]:
assistant = assistant_client.beta.assistants.create(
    name="Sales Assistant",
    instructions="You are a sales assistant. You can answer questions related to customer orders.",
    tools=tools_list,
    model=api_deployment_name,
    file_ids=file_ids,
)
sales.ai_assistants.append(assistant)

### Go to load balance endpoint or to the Assistant based on intent

In [ ]:
def process_on_intent(user: str, user_id: str, prompt: str):
    # Poor man's Semantic Analysis (for now)
    strings = ['calculate', 'chart', 'display','customers','sold']    
    if any(s in prompt.lower() for s in strings):
        sales.process_prompt(assistant_client, assistant, user, user_id, "What customers are in Florida?")
    else:
        completion = load_balanced_client.chat.completions.create(
            model=api_deployment_name,
            messages=[
                {
                    "role": "user",
                    "content": prompt,
                },
            ],
        )
        print(completion.choices[0].message.content)

### Have a conversation with the Assistant

In [ ]:
process_on_intent("Suzan", "user_123", "What is the speed of light?")

In [ ]:
process_on_intent("John", "user_123", "What customers are in Florida?")

In [ ]:
process_on_intent("Mary", "user_234", "What seller has had the most sales?")

In [ ]:
process_on_intent("John", "user_123", "What is the most sold product?")

In [ ]:
process_on_intent("Mary", "user_234", "Chart product sales by State.")

### Cleanup

In [ ]:
sales.cleanup(assistant_client)